In [ ]:
hittingTeams = pd.read_csv("Baseball_Hitting_Team_2023.csv")
#hittingTeams.head()

pitchingPlayers = pd.read_csv("Baseball_Pitching_Player_2023.csv")
#pitchingPlayers.head()

pitchingTeams = pd.read_csv("Baseball_Pitching_Team_2023.csv")
#pitchingTeams.head()

In [ ]:
# Pitchers

# Drop unneeded columns
pitchingPlayers = pitchingPlayers[['Name', 'Tm', 'IP', 'H9', 'ERA', 'HR9', 'BB9', 'SO9']]
pitchingPlayers["Name"] = pitchingPlayers["Name"].astype(str)

pitchingPlayers = pitchingPlayers.dropna()
# Clean player names
pitchingPlayers['Name'] = pitchingPlayers['Name'].str.replace('*', '', regex=False)
pitchingPlayers['Name'] = pitchingPlayers['Name'].str.replace('#', '', regex=False)
#Set min ABs
minIPs = 100
pitchingPlayers = pitchingPlayers[pitchingPlayers['IP'] > minIPs]
pitchingPlayers = pitchingPlayers.drop(columns=['IP'])
pitchingPlayers = pitchingPlayers[pitchingPlayers['Tm'] != 'TOT']
#pitchingPlayers.head()

In [ ]:
Data Manipulation
Hitters
#Create Value Metric
def hitterValue(hittingPlayers, slices):
    runs= hittingPlayers['R']
    hits= hittingPlayers['H']
    HRs= hittingPlayers['HR']
    RBIs= hittingPlayers['RBI']
    meanRuns = runs.mean()
    meanHits = hits.mean()
    meanHRs = HRs.mean()
    meanRBIs = RBIs.mean()
    hittingPlayers['Value'] = ((hits - meanHits)/meanHits)*slices[1]  + ((runs - meanRuns)/meanRuns)*slices[2] + ((hits - meanRBIs)/meanRBIs)*slices[3] + ((HRs - meanHRs)/meanHRs)*slices[4]
    hittingPlayers['Value'] = hittingPlayers['Value'] / hittingPlayers['Value'].max() #Normalization
    hittingPlayers = hittingPlayers.sort_values('Value', ascending = False).reset_index(drop=True)
    return hittingPlayers
# Highest value player on each team
def teamBestHitter(hittingPlayers):
    idxmax_values = hittingPlayers.groupby('Tm')['Value'].idxmax()
    highest_valued_players = hittingPlayers.loc[idxmax_values].set_index("Tm",drop=True)
    return highest_valued_players
topRanks
#Attach team wins to valueChart
def hittingWins(hittingPlayers, topNMVPs, standings):
    valueChart = pd.DataFrame()
    for numTopPlayers in range(1, topNMVPs + 1):
        valueChart[f'Rank{numTopPlayers}'] = hittingPlayers.groupby('Tm')['Value'].nlargest(numTopPlayers).groupby(level=0).mean()
    topRanks = pd.DataFrame()
    rank_columns = [f'{topNMVPs}' for topNMVPs in valueChart.columns]
    topRanks[rank_columns] = valueChart.rank(axis=0, method='min', ascending = False).astype(int)
    hittingWins = pd.merge(standings['W'], topRanks, on='Tm')
    valueChart = pd.merge(standings['W'], valueChart, on='Tm')
    return hittingWins, valueChart
# Divide by column average
#weightedTopPlayerValues = pd.DataFrame()   
# Run all data manipulation functions for hitting data
# def hitterDataManip(hittingPlayers, topNMVPs, standings):
#     hittingPlayers = hitterValue(hittingPlayers)
#     hittingWins = hittingWins(hittingPlayers, topNMVPs, standings)
#     highest_valued_players = teamBestHitter(hittingPlayers)
#     #valueChart = valueChart(hittingPlayers, topNMVPs)
#     grouped_means = (
#         hittingPlayers.groupby('Tm')['Value']
#         .nlargest(topNMVPs)
#         .groupby(level=0)
#         .mean()
#         .reset_index()
#     )
#     return (hittingPlayers, highest_valued_players, hittingWins)


In [ ]:
#Create Value Metric

# hits= pitchingPlayers['H9']
# ERAs= pitchingPlayers['ERA']
# HRs= pitchingPlayers['HR9']
# BBs= pitchingPlayers['BB9']
# SOs= pitchingPlayers['SO9']


# meanHits = hits.mean()
# meanERAs = ERAs.mean()
# meanHRs = HRs.mean()S
# meanBBs = BBs.mean()
# meanSOs = SOs.mean()

# pitchingPlayers['Value'] = -(((hits - meanHits)/meanHits)*.2  + ((BBs - meanBBs)/meanBBs)*.2 + ((ERAs - meanERAs)/meanERAs)*.3 - ((SOs - meanSOs)/meanSOs)*.1 + ((HRs - meanHRs)/meanHRs)*.1)
# pitchingPlayers['Value'] = pitchingPlayers['Value'] / pitchingPlayers['Value'].max()
# pitchingPlayers = pitchingPlayers.sort_values('Value', ascending = False).reset_index(drop=True)
#pitchingPlayers

# Highest value player on each team

# idxmax_values = pitchingPlayers.groupby('Tm')['Value'].idxmax()
# highest_valued_pitchers = pitchingPlayers.loc[idxmax_values].set_index("Tm",drop=True)
#highest_valued_pitchers.head()

In [ ]:
# Assuming X is your feature matrix and y is your target variable
# Replace X and y with your actual data
def createPlayerWinPredGraphs(valueChart, numPlayers):
    X_train = [None] * (len(numPlayers) + 1)
    X_test = [None] * (len(numPlayers) + 1)
    y_train = [None] * (len(numPlayers) + 1)
    y_test = [None] * (len(numPlayers) + 1)
    y_pred = [None] * (len(numPlayers) + 1)
    r2 = [None] * (len(numPlayers) + 1)
    for numPlayer in range(0,len(numPlayers)):
        X = valueChart[[f'Rank{numPlayers[numPlayer]}']]
        y = valueChart['W']

        # Split the data into training and testing sets with a test size of 20%
        X_train[numPlayer], X_test[numPlayer], y_train[numPlayer], y_test[numPlayer] = train_test_split(X, y, test_size=0.1, random_state=42, stratify=valueChart['Yr'])

        # Check the sizes of the resulting sets
#         print("Training set size:", len(X_train[numPlayer]))
#         print("Testing set size:", len(X_test[numPlayer]))
    
        # Fit the model on the training set
#         model = LinearRegression(random_state=42)
#         model = RandomForestRegressor(n_estimators=100, random_state=42)
        model = GradientBoostingRegressor(n_estimators=1, random_state=42)
        model.fit(X_train[numPlayer], y_train[numPlayer])
#         print(X_test)
        X_test[numPlayer] = X_test[numPlayer].sort_values(by=f'Rank{numPlayers[numPlayer]}', ascending=True)
        # Make predictions on the test set
        y_pred[numPlayer] = model.predict(X_test[numPlayer])
        # Calculate R-squared

#         print(y_test)
        r2[numPlayer] = r2_score(y_test[numPlayer], y_pred[numPlayer])
#         print(f'R-squared for ({numPlayers[numPlayer]}p): {r2}')
        
    return X_train, X_test , y_train, y_test, y_pred, r2

In [ ]:
def plot_scatter(typePlayer,ax, X, y, result_numbers):
    X[0] = X[0].reset_index(drop = True)
#     print(len(numPlayers))
    for numPlayer in range(len(numPlayers)):
        ax.scatter(X[numPlayer], y[numPlayer], label=f'Top {numPlayer + 1}P')
        ax.axvline(x=np.mean(X[numPlayer]), label='50th Percentile')
        if numPlayer > 0:
            for team in range(len(X[1])):
                ax.plot([X[numPlayer-1].iloc[team][0], X[numPlayer].iloc[team][0]], [y[numPlayer-1].iloc[team], y[numPlayer].iloc[team]], color='gray', linestyle='--')        

    ax.set_xlabel(f'Top {numPlayers} Players Avg. Value (Normalized)')
    ax.set_ylabel('Team Wins')
    ax.legend(loc='lower right')
    if typePlayer == "Hitter":
        ax.set_title(f'Hits={result_numbers[1]}%, Runs={result_numbers[2]}%, HRs={result_numbers[3]}%, Ks={result_numbers[4]}%')
    else:
        ax.set_title(f'H9={result_numbers[1]}%, ERA={result_numbers[2]}%, HR9={result_numbers[3]}%, SO9={result_numbers[4]}%')


In [ ]:
numPlayers = [1,9]

fig, axs = plt.subplots(2, 2, figsize=(15, 15))    
    
valueChartH, valueChartP, result_numbers = createCompilations(52, [0,50,25,25,0], numPlayers, standings23, standings22, standings21, standings19, standings18, standings17, standings16, standings15, standings14, standings13, hittingPlayers23, hittingPlayers22, hittingPlayers21, hittingPlayers19, hittingPlayers18, hittingPlayers17, hittingPlayers16, hittingPlayers15, hittingPlayers14, hittingPlayers13, pitchingPlayers23, pitchingPlayers22, pitchingPlayers21, pitchingPlayers19, pitchingPlayers18, pitchingPlayers17, pitchingPlayers16, pitchingPlayers15, pitchingPlayers14, pitchingPlayers13)
X_train, X_test, y_train, y_test, y_pred, r2 = createPlayerWinPredGraphs(valueChartH, numPlayers)
plot_scatter("Hitter", axs[0, 0], X_test, y_test, [0,50,25,25,0])
axs[0, 0].set_xlim(0, 1)
axs[0, 0].set_ylim(35, 115)

valueChartH, valueChartP, result_numbers = createCompilations(34, [0,20,50,20,10], numPlayers, standings23, standings22, standings21, standings19, standings18, standings17, standings16, standings15, standings14, standings13, hittingPlayers23, hittingPlayers22, hittingPlayers21, hittingPlayers19, hittingPlayers18, hittingPlayers17, hittingPlayers16, hittingPlayers15, hittingPlayers14, hittingPlayers13, pitchingPlayers23, pitchingPlayers22, pitchingPlayers21, pitchingPlayers19, pitchingPlayers18, pitchingPlayers17, pitchingPlayers16, pitchingPlayers15, pitchingPlayers14, pitchingPlayers13)
X_train, X_test, y_train, y_test, y_pred, r2 = createPlayerWinPredGraphs(valueChartH, numPlayers)
plot_scatter("Hitter", axs[1, 0], X_test, y_test, [0,20,50,20,10])
axs[1, 0].set_xlim(0, 1)
axs[1, 0].set_ylim(35, 115)

valueChartH, valueChartP, result_numbers = createCompilations(10, [0,0,100,0,0], numPlayers, standings23, standings22, standings21, standings19, standings18, standings17, standings16, standings15, standings14, standings13, hittingPlayers23, hittingPlayers22, hittingPlayers21, hittingPlayers19, hittingPlayers18, hittingPlayers17, hittingPlayers16, hittingPlayers15, hittingPlayers14, hittingPlayers13, pitchingPlayers23, pitchingPlayers22, pitchingPlayers21, pitchingPlayers19, pitchingPlayers18, pitchingPlayers17, pitchingPlayers16, pitchingPlayers15, pitchingPlayers14, pitchingPlayers13)
X_train, X_test, y_train, y_test, y_pred, r2 = createPlayerWinPredGraphs(valueChartH, numPlayers)
plot_scatter("Hitter", axs[0, 1], X_test, y_test, [0,0,100,0,0])
axs[0, 1].set_xlim(0, 1)
axs[0, 1].set_ylim(35, 115)

valueChartH, valueChartP, result_numbers = createCompilations(11, [0,0,50,0,50], numPlayers, standings23, standings22, standings21, standings19, standings18, standings17, standings16, standings15, standings14, standings13, hittingPlayers23, hittingPlayers22, hittingPlayers21, hittingPlayers19, hittingPlayers18, hittingPlayers17, hittingPlayers16, hittingPlayers15, hittingPlayers14, hittingPlayers13, pitchingPlayers23, pitchingPlayers22, pitchingPlayers21, pitchingPlayers19, pitchingPlayers18, pitchingPlayers17, pitchingPlayers16, pitchingPlayers15, pitchingPlayers14, pitchingPlayers13)
X_train, X_test, y_train, y_test, y_pred, r2 = createPlayerWinPredGraphs(valueChartH, numPlayers)
plot_scatter("Hitter", axs[1, 1], X_test, y_test, [0,0,50,0,50])
axs[1, 1].set_xlim(0, 1)
axs[1, 1].set_ylim(35, 115)

plt.tight_layout()
plt.show()

In [ ]:
numPlayers = [1,5]

fig, axs = plt.subplots(2, 2, figsize=(15, 15))    
    
valueChartH, valueChartP, result_numbers = createCompilations(1, [0,50,25,25,0], numPlayers, standings23, standings22, standings21, standings19, standings18, standings17, standings16, standings15, standings14, standings13, hittingPlayers23, hittingPlayers22, hittingPlayers21, hittingPlayers19, hittingPlayers18, hittingPlayers17, hittingPlayers16, hittingPlayers15, hittingPlayers14, hittingPlayers13, pitchingPlayers23, pitchingPlayers22, pitchingPlayers21, pitchingPlayers19, pitchingPlayers18, pitchingPlayers17, pitchingPlayers16, pitchingPlayers15, pitchingPlayers14, pitchingPlayers13)
X_train, X_test, y_train, y_test, y_pred, r2 = createPlayerWinPredGraphs(valueChartP, numPlayers)
plot_scatter("Pitcher", axs[0, 0], X_test, y_test, [0,50,25,25,0])

valueChartH, valueChartP, result_numbers = createCompilations(100, [0,20,50,20,10], numPlayers, standings23, standings22, standings21, standings19, standings18, standings17, standings16, standings15, standings14, standings13, hittingPlayers23, hittingPlayers22, hittingPlayers21, hittingPlayers19, hittingPlayers18, hittingPlayers17, hittingPlayers16, hittingPlayers15, hittingPlayers14, hittingPlayers13, pitchingPlayers23, pitchingPlayers22, pitchingPlayers21, pitchingPlayers19, pitchingPlayers18, pitchingPlayers17, pitchingPlayers16, pitchingPlayers15, pitchingPlayers14, pitchingPlayers13)
X_train, X_test, y_train, y_test, y_pred, r2 = createPlayerWinPredGraphs(valueChartP, numPlayers)
plot_scatter("Pitcher", axs[1, 0], X_test, y_test, [0,20,50,20,10])

valueChartH, valueChartP, result_numbers = createCompilations(42, [0,0,100,0,0], numPlayers, standings23, standings22, standings21, standings19, standings18, standings17, standings16, standings15, standings14, standings13, hittingPlayers23, hittingPlayers22, hittingPlayers21, hittingPlayers19, hittingPlayers18, hittingPlayers17, hittingPlayers16, hittingPlayers15, hittingPlayers14, hittingPlayers13, pitchingPlayers23, pitchingPlayers22, pitchingPlayers21, pitchingPlayers19, pitchingPlayers18, pitchingPlayers17, pitchingPlayers16, pitchingPlayers15, pitchingPlayers14, pitchingPlayers13)
X_train, X_test, y_train, y_test, y_pred, r2 = createPlayerWinPredGraphs(valueChartP, numPlayers)
plot_scatter("Pitcher", axs[0, 1], X_test, y_test, [0,0,100,0,0])

valueChartH, valueChartP, result_numbers = createCompilations(123, [0,0,50,0,50], numPlayers, standings23, standings22, standings21, standings19, standings18, standings17, standings16, standings15, standings14, standings13, hittingPlayers23, hittingPlayers22, hittingPlayers21, hittingPlayers19, hittingPlayers18, hittingPlayers17, hittingPlayers16, hittingPlayers15, hittingPlayers14, hittingPlayers13, pitchingPlayers23, pitchingPlayers22, pitchingPlayers21, pitchingPlayers19, pitchingPlayers18, pitchingPlayers17, pitchingPlayers16, pitchingPlayers15, pitchingPlayers14, pitchingPlayers13)
X_train, X_test, y_train, y_test, y_pred, r2 = createPlayerWinPredGraphs(valueChartP, numPlayers)
plot_scatter("Pitcher", axs[1, 1], X_test, y_test, [0,0,50,0,50])

plt.tight_layout()
plt.show()

In [ ]:
X = hittingWins['Rank1']
y = hittingWins['W']

# Split the data into training and testing sets with a test size of 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the sizes of the resulting sets
print("Training set size:", len(X_train))
print("Testing set size:", len(X_test))

model = LinearRegression()

# Fit the model on the training set
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred

In [ ]:
import array
result_numbers = [0]
sum(result_numbers)
random_number = np.random.randint(1, 101-sum(result_numbers),1)
result_numbers = result_numbers.append(random_number[0])
print(result_numbers)
result_numbers

In [ ]:
#plt.plot(result_df, result_df['Team'])
df_transposed = valueChart.transpose()
for team in df_transposed.columns:
    plt.plot(df_transposed.index, df_transposed[team], marker='o', label=team)

In [ ]:
def findDropOff(df):
    slopes = []
    for team in df.index:
        x = np.arange(1, len(df.columns) + 1)
        y = df.loc[team].values
        slope, _ = np.polyfit(x, y, 1)
        slopes.append(slope)
    df['DropOff'] = slopes
    
findDropOff(valueChart)
valueChart.head()